### **DBSCAN**


#### **Importaciones**

In [ ]:
import heapq
import math
import matplotlib.colors as mcolors
from matplotlib import pyplot as plt

from collections import Counter
from itertools import chain, groupby
from random import randrange, random
from scipy.spatial import KDTree
from typing import List, Tuple

#### **El algoritmo**

In [ ]:
NOISE = -1  # Constante para marcar puntos considerados ruido

In [ ]:
def dbscan(points: List[Tuple[float, float]], eps: float, min_points: int) -> List[int]:
    """
    Implementación básica de DBSCAN.
    points: lista de tuplas (x, y)
    eps: radio de vecindad
    min_points: número mínimo de puntos para considerar un núcleo
    Devuelve una lista de etiquetas de clúster (int) para cada punto.
    """
    n = len(points)
    # Inicializamos todas las etiquetas como None (sin clasificar)
    cluster_indices = [None] * n
    # Contador de clústeres (comenzará en 1)
    current_index = 0
    # Construimos un KD-Tree para acelerar búsquedas de vecinos
    kd_tree = KDTree(points)

    # Recorremos cada punto
    for i in range(n):
        # Si ya fue asignado a un clúster o marcado como ruido, lo saltamos
        if cluster_indices[i] is not None:
            continue

        # Iniciamos un nuevo conjunto de proceso con el punto i
        process_set = {i}
        # Marcamos provisionalmente el punto como ruido (podría cambiar si es núcleo)
        cluster_indices[i] = NOISE
        # Avanzamos al siguiente índice de clúster
        current_index += 1

        # Exploramos todos los puntos en el conjunto de proceso
        while process_set:
            j = process_set.pop()
            # Buscamos vecinos de j dentro del radio eps
            neighbors = kd_tree.query_ball_point(points[j], eps)

            # Si no alcanza min_points, j no es núcleo; descartamos
            if len(neighbors) < min_points:
                continue

            # j es un punto de núcleo: lo asignamos al clúster actual
            cluster_indices[j] = current_index

            # Revisamos cada vecino para expandir el clúster
            for p in neighbors:
                # Si p nunca fue visitado (None) o estaba marcado como ruido, lo añadimos
                if cluster_indices[p] is None or (p != j and cluster_indices[p] == NOISE):
                    process_set.add(p)

    return cluster_indices


#### **Creación de datos**

In [ ]:
def create_spherical_cluster(centroid, radius, n_points):
    def random_point_in_circle():
        alpha = random() * 2 * math.pi
        r = radius * math.sqrt(random())
        x = centroid[0] + r * math.cos(alpha)
        y = centroid[1] + r * math.sin(alpha)
        return (x, y)
    return [random_point_in_circle() for _ in range(n_points)]

In [ ]:
def create_ring_cluster(centroid, inner_radius, outer_radius, n_points):
    def random_point_in_ring():
        alpha = random() * 2 * math.pi
        r = inner_radius + (outer_radius - inner_radius) * math.sqrt(random())
        x = centroid[0] + r * math.cos(alpha)
        y = centroid[1] + r * math.sin(alpha)
        return (x, y)
    return [random_point_in_ring() for _ in range(n_points)]

In [ ]:
def create_linear_cluster(centroid, length, n_points, alpha=0):
    delta = length / n_points
    step = 0.0
    def random_point_in_segment(step, displacement_x):
        x = centroid[0] + step * math.cos(alpha) + displacement_x
        y = centroid[1] + step * math.sin(alpha)
        return (x, y)
    points = []
    for i in range(n_points):
        step += (0.5 + random() / 2) * delta
        for _ in range(randrange(1, 10)):
            displacement_x = random() * delta * 2 - delta
            points.append(random_point_in_segment(step, displacement_x))
    return points

In [ ]:
def create_spiral_cluster(centroid, radius, n_points):
    a_delta = 10 * math.pi / n_points
    r_delta = radius / n_points
    def random_point_in_spiral(index, alpha):
        x = centroid[0] + index * r_delta * (0.95 + random() * 0.1) * math.cos(alpha)
        y = centroid[1] + index * r_delta * (0.95 + random() * 0.1) * math.sin(alpha)
        return (x, y)
    alpha = 0.0
    points = []
    for i in range(n_points):
        step = 0.1 + random() * 0.4
        alpha += a_delta * step
        for _ in range(randrange(1, 10)):
            points.append(random_point_in_spiral(i, alpha))
            step = random() * 0.1
            alpha += a_delta * step
    return points

In [ ]:
def random_point(radius):
    return (random() * radius - radius / 2, random() * radius - radius / 2)

#### **Visualización**

In [ ]:
def __group_points_by_cluster(points: List[Tuple[float, float]], labels: List[int]) -> List[List[Tuple[float, float]]]:
    n = len(points)
    ordered = sorted(range(n), key=lambda i: labels[i])
    clusters = groupby(ordered, key=lambda i: labels[i])
    return [[points[i] for i in group] for _, group in clusters]

In [ ]:
def plot_points(ax, points, color, marker='o'):
    X, Y = zip(*points)
    ax.plot(X, Y, marker=marker, linestyle='none',
            markersize=8, markerfacecolor=color,
            markeredgewidth=1, markeredgecolor='k')

def plot_clusters(points, labels, point_markers=None, title='DBSCAN', ax=None):
    if point_markers is None:
        point_markers = []
    clusters = __group_points_by_cluster(points, labels)
    n_clusters = len(clusters)

    # Definir paleta de colores
    if NOISE in labels:
        base_colors = ['k'] + list(mcolors.TABLEAU_COLORS.values())
    else:
        base_colors = list(mcolors.TABLEAU_COLORS.values())

    # Asegurarnos de tener suficientes colores
    colors = [base_colors[i % len(base_colors)] for i in range(n_clusters)]
    # Definir marcadores: si no alcanza, reutilizamos 'o'
    markers = [point_markers[i] if i < len(point_markers) else 'o' for i in range(n_clusters)]

    if ax is None:
        fig, ax = plt.subplots()
        fig.suptitle(title)
        ax.set_aspect('equal')
    for i, cluster in enumerate(clusters):
        plot_points(ax, cluster, colors[i], markers[i])
    plt.show()


### **Experimentos**


**Clusters esféricos**

In [ ]:
# Generar datos de prueba
k = 10
data_centroids = [random_point(10) for _ in range(k)]
data_clusters = [create_spherical_cluster(P, 0.25 + random(), randrange(10, 100)) for P in data_centroids]
noise = [random_point(10) for _ in range(50)]
points = [p for C in data_clusters for p in C] + noise

In [ ]:
labels = dbscan(points, eps=0.5, min_points=2)
plot_clusters(points, labels)

#### **Anillos**

In [ ]:
centroid = random_point(10)
r1 = 0.25 + random(); r2 = r1 + 0.25 * random()
r3 = r2 * 1.1 + random(); r4 = r3 + 0.25 * random()
C1 = create_ring_cluster(centroid, r1, r2, randrange(100, 200))
C2 = create_ring_cluster(centroid, r3, r4, randrange(200, 300))
points = C1 + C2

In [ ]:
labels1 = dbscan(points, eps=0.5, min_points=3)

In [ ]:
plot_clusters(points, labels1, ['P', 's'])

In [ ]:
labels2 = dbscan(points, eps=0.1, min_points=3)

In [ ]:
plot_clusters(points, labels2, ['x', 's', '*'])

**Espirales**

In [ ]:
C3 = create_spiral_cluster(random_point(10), 5, 100)

In [ ]:
labels3 = dbscan(C3, eps=1, min_points=3)

In [ ]:
plot_clusters(C3, labels3, ['o', 's', 'P'])

**Segmentos**

In [ ]:
P4 = random_point(10)
alpha4 = random() * math.pi / 2
C4 = create_linear_cluster(P4, 5, 100, alpha4) + create_linear_cluster((P4[0] + 1, P4[1]), 4, 100, alpha4 + 0.1)

In [ ]:
labels4 = dbscan(C4, eps=0.5, min_points=3)  

In [ ]:
plot_clusters(C4, labels4, ['o', 's'])

**Densidad**

In [ ]:
C5 = create_spherical_cluster((-2,0), 2, 50) + create_spherical_cluster((2,2), 0.5, 20) + create_spherical_cluster((2,0.78), 0.25, 10)

In [ ]:
labels_5 = dbscan(C5, 0.5, 3)

In [ ]:
plot_clusters(C5, labels_5, ['s', 'o', 'p', '*', 'h', 'H', 'X', 'D', 'd'])

In [ ]:
print(labels_5)

In [ ]:
labels_5b = dbscan(C5, 1, 3)
plot_clusters(C5, labels_5b, ['s', 'o', 'p', '*', 'h', 'H', 'X', 'D', 'd'])

In [ ]:
labels_5b = dbscan(C5, 0.69875, 3)
plot_clusters(C5, labels_5b, ['s', 'o', 'p', '*', 'h', 'H', 'X', 'D', 'd'])

#### **Ejercicios**

1. **Tuning de parámetros para distintos formatos de cluster**

   * Genera datos usando `create_spherical_cluster`, `create_ring_cluster`, `create_spiral_cluster` y `create_linear_cluster`.
   * Para cada tipo de cluster, realiza un barrido (grid search) de valores de `eps` y `min_points`.
   * Documenta gráficamente cómo cambia el número de clusters identificados y la proporción de puntos etiquetados como ruido.

2. **Detección de densidades variables**

   * Crea un dataset que mezcle regiones de alta densidad (pequeño radio, muchos puntos) y baja densidad (radio grande, pocos puntos).
   * Ajusta los parámetros de DBSCAN de forma estática y observa dónde falla la detección.
   * Propón y prueba una estrategia para usar distintos valores de `eps` en función de la densidad local (por ejemplo, clustering jerárquico previo o estimación de distancias k-ésimas).

3. **Comparación con otros algoritmos de clustering**

   * Implementa o usa `sklearn.cluster.KMeans` y `sklearn.cluster.AgglomerativeClustering` sobre el mismo dataset sintético.
   * Compara resultados cualitativos (gráficos) y cuantitativos (e.g. índice de silhouette, Davies–Bouldin) frente a DBSCAN.

4. **Etiquetado de puntos de "borde" y "núcleo"**

   * Modifica la función `dbscan` (o crea una nueva) para que devuelva no solo la etiqueta de cluster, sino también si cada punto es de tipo **core**, **border** o **noise**.
   * Visualiza con distintos marcadores (círculo relleno, círculo vacío, cruz) cada uno de estos tipos.

5. **DBSCAN en 3D**

   * Extiende el código para trabajar con puntos en (tuplas `(x,y,z)`).
   * Genera un cluster esférico (`create_spherical_cluster`) y una "espiral 3D" personalizada.
   * Visualiza los resultados usando `matplotlib` con proyección 3D.

6. **Eficiencia y complejidad**

   * Mide el tiempo de ejecución de `dbscan` para distintos tamaños de dataset (p.ej. 1 000; 10 000; 100 000 puntos).
   * Compara tiempos usando KD-Tree, brute force y, opcionalmente, un Ball-Tree de `scipy`.
   * Grafica la complejidad empírica (tiempo vs. número de puntos) y discute.

7. **Clustering de datos reales**

   * Carga un dataset real en 2D (por ejemplo, coordenadas geográficas de ciudades, o características PCA de Iris).
   * Aplica DBSCAN, elige parámetros adecuados y comenta los clusters que se forman (p. ej. zonas urbanas vs. rurales).

8. **Implementación de HDBSCAN ligera**

   * Investiga brevemente el algoritmo HDBSCAN (clustering jerárquico basado en densidad).
   * Utiliza la librería `hdbscan` de Python para comparar resultados con DBSCAN en datasets sintéticos de anillos y espirales.

9. **Visualización interactiva**

   * Crea una interfaz sencilla (por ejemplo, con `ipywidgets`) que permita ajustar `eps` y `min_points` mediante sliders y vea en tiempo real el plot de clusters.
   * Añade un selector de tipo de dataset (`spherical`, `ring`, `spiral`, `linear`).

10. **Evaluación de calidad de clusters**

    * Para cada resultado de DBSCAN, calcula métricas como **Silhouette Score**, **Calinski–Harabasz** y **Davies–Bouldin**.
    * Analiza cómo varían estas métricas al cambiar `eps` y `min_points`, y usa tus conclusiones para recomendar reglas empíricas de parametrización.

11. **DBSCAN sobre streaming**

    * Diseña un esquema para procesar puntos que llegan en tiempo real (streaming).
    * Plantea cómo añadir/eliminar puntos del clustering sin recomputar todo desde cero (algoritmo incremental).



In [ ]:
## Tus respuestas